ProblemStatement : We want to predict the energy output in Megawatt

- Temperature (T) in the range 1.81°C and 37.11°C,
- Ambient Pressure (AP) in the range 992.89-1033.30 milibar,
- Relative Humidity (RH) in the range 25.56% to 100.16%
- Exhaust Vacuum (V) in teh range 25.36-81.56 cm Hg
- Net hourly electrical energy output (PE) 420.26-495.76 MW

In [76]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [77]:
df = pd.read_excel('/content/Folds5x2_pp.xlsx')

In [78]:
df.head(3)

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56


# EDA Section

In [79]:
df.describe()

,AT,V,AP,RH,PE
count,9568.000000,9568.000000,9568.000000,9568.000000,9568.000000
mean,19.651231,54.305804,1013.259078,73.308978,454.365009
std,7.452473,12.707893,5.938784,14.600269,17.066995
min,1.810000,25.360000,992.890000,25.560000,420.260000
25%,13.510000,41.740000,1009.100000,63.327500,439.750000
50%,20.345000,52.080000,1012.940000,74.975000,451.550000
75%,25.720000,66.540000,1017.260000,84.830000,468.430000
max,37.110000,81.560000,1033.300000,100.160000,495.760000


Intepretation:

AT: Ambient Temperature
1. Mean 19.65
2. Range between 1.81 which is the lowest to 37.1 which is the highest temperature
3. The power plant operates in a wide range of temperatures. A median of 20.35°C suggests most observations are close to this value.

V: Exhaust Vacuum Pressure
1. Mean: 54.31 cm Hg
2. Range: 25.36 cm Hg to 81.56 cm Hg
3. Vacuum pressure varies significantly, indicating operational changes or environmental factors affecting the system.

AP: Ambient Pressure
1. Mean: ~1013.26 mbar
2. Range: 992.89 mbar to 1033.3 mbar
3. Ambient pressure remains fairly stable with a small standard deviation of 5.94 mbar, which may still impact power output (PE).

RH: Relative Humidity
1. Mean: ~73.31%
2. Range: 25.56% to 100.16%
3. Relative humidity varies widely, potentially influencing the cooling process and energy efficiency.

PE Net Hourly Electrical Output:
1. Mean: ~454.37 MW
2. Range: 420.26 MW to 495.76 MW
3. Power output varies by ~76 MW, likely driven by the other factors in the dataset.

Ambient Temperature (AT) might negatively correlate with power output (PE), as higher temperatures reduce thermal efficiency.

Relative Humidity (RH) might also negatively affect PE, as higher humidity can hinder cooling.

Ambient Pressure (AP) and Exhaust Vacuum (V) may have subtler but meaningful relationships with PE.

# Correlation Matrix

In [80]:
df.corr()

,AT,V,AP,RH,PE
AT,1.000000,0.844107,-0.507549,-0.542535,-0.948128
V,0.844107,1.000000,-0.413502,-0.312187,-0.869780
AP,-0.507549,-0.413502,1.000000,0.099574,0.518429
RH,-0.542535,-0.312187,0.099574,1.000000,0.389794
PE,-0.948128,-0.869780,0.518429,0.389794,1.000000


Intepretation:
1. The AT value has a high correlation with Exhaust Vaccum.
2. The AP value has a positive correlation with PE

Separate the features from the labels

In [81]:
# I want all the row and columns except the last column
X = df.iloc[:, :-1].values

In [82]:
# I want all the rows but my label is the last column
y = df.iloc[:, -1].values

Split in test and train

In [83]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)

For the training of the neural network I will
1. Create the tensors in Temperature, Pressure, Humidity, Vaccuum then I will add some layers to process the data and the output layer will give me the Energy output

In [84]:
## Initialize the neural network
ann = tf.keras.models.Sequential()

## Adding a hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## Adding a second hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## Adding the output layer
ann.add(tf.keras.layers.Dense(units=1))


Compile the ann

In [85]:
ann.compile(optimizer='adam', loss='mean_squared_error')

Train the model with the training data

In [86]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 20)

Epoch 1/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 82684.2500
Epoch 2/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 276.8878
Epoch 3/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 267.8154
Epoch 4/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 246.9649
Epoch 5/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 231.6694
Epoch 6/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 211.8257
Epoch 7/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 190.6313
Epoch 8/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 168.3820
Epoch 9/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 147.9589
Epoch 10/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 129.0683
Epoch 11/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 108.7293
Epoch 12/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 92.6694
Epoch 13/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 74.5737
Epoch 14/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 62.5015
Epoch 15/20
255/255 ━━━━━━━━━━

Create predictions

In [87]:
y_pred = ann.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[[430.42 431.23]
 [465.42 460.01]
 [471.71 461.14]
 ...
 [447.67 445.6 ]
 [460.51 451.7 ]
 [460.49 460.45]]


Implement transfer learning architecture I will try to use

A little bit about the resnet:
The Bayesian regression ResNet is built upon the regression ResNet, and dropout layers are inserted between residual blocks.

In [88]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout, Add, BatchNormalization, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

Reload the data to for the Resnet transfer learning model

In [89]:
df = pd.read_excel("Folds5x2_pp.xlsx")

Separate the features from the labels

In [90]:
# Separate features and target
X = df.drop(columns=['PE'])
y = df['PE']

Make the train and test split

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Use a standard scaler to Standardization
Definition: Standardization transforms the data to have a mean of 0 and a standard deviation of 1.

In [92]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Apply the resnet architecture

Dropout rate:  Learning rate dropout (LRD) is a new gradient descent technique to motivate faster convergence and better generalization.

Convergence:

In deep learning, convergence refers to the point at which the training process reaches a stable state and the parameters of the network (i.e., the weights and biases) have settled on values that produce accurate predictions for the training data


In [93]:
def resnet_block(inputs, units, dropout_rate=0.2):
    """
    A residual block for tabular data. Using the resnet algorithm to predict the Net Electrical Output PE
    Args:
        inputs: Input tensor (Data that is transformed to a tensor)
        units: Number of neurons in the dense layers
        dropout_rate: Dropout rate
    Returns:
        Output tensor
    """
    # Shortcut connection
    shortcut = inputs

    # Dense Layer 1
    x = Dense(units, activation=None)(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(dropout_rate)(x)

    # Dense Layer 2
    x = Dense(units, activation=None)(x)
    x = BatchNormalization()(x)

    # Add shortcut connection
    x = Add()([x, shortcut])
    x = ReLU()(x)

    return x

Build the resnet model and check the model performace

In [94]:
# Build the ResNet model
def build_resnet_model(input_shape):
    """
    Builds a ResNet-style architecture for tabular data regression.
    Args:
        input_shape: Shape of the input features
    Returns:
        ResNet model
    """
    inputs = Input(shape=(input_shape,))

    # Initial Dense Layer
    x = Dense(128, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    # Residual Blocks
    for _ in range(3):  # Add three residual blocks
        x = resnet_block(x, units=128)

    # Output Layer
    outputs = Dense(1, activation='linear')(x)

    # Define the model
    model = Model(inputs=inputs, outputs=outputs)

    return model

Substep 1: Create the mode

In [95]:
resnet_model = build_resnet_model(X_train.shape[1])

Substep 2: Compile the model

In [96]:
resnet_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

Add early stopping

Definition :Early Stopping is a regularization technique for deep neural networks that stops training when parameter updates no longer begin to yield improves on a validation set.


In [97]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


Substep 4: Train the model on the data



In [98]:
model_trained = resnet_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 195053.9531 - mae: 441.1217 - val_loss: 164655.3594 - val_mae: 405.5123
Epoch 2/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 137006.7188 - mae: 369.0101 - val_loss: 88118.7812 - val_mae: 296.5016
Epoch 3/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 62118.5820 - mae: 246.4479 - val_loss: 34584.9766 - val_mae: 184.9783
Epoch 4/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 13262.2373 - mae: 110.8176 - val_loss: 6659.8940 - val_mae: 80.9892
Epoch 5/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1162.2687 - mae: 29.7520 - val_loss: 1258.2220 - val_mae: 34.0153
Epoch 6/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 180.7916 - mae: 10.3866 - val_loss: 418.9642 - val_mae: 19.6521
Epoch 7/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 110.6068 - mae: 8.1745 - val_loss: 237.6035 - val_mae: 14.1189
Epoch 8/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 97.9979 - mae: 7.7206 - val_loss: 92.1263 - va

In [99]:
# Evaluate the model
test_loss, test_mae = resnet_model.evaluate(X_test, y_test, verbose=0)
print(f"Test MAE: {test_mae}")

Test MAE: 4.538840293884277


Currently that means that on average the predictions of the model are 4.042 points away which is fairly good

Make prediction on the new model using the test set

In [100]:
# Predict on new data
predictions = resnet_model.predict(X_test)

75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [101]:
y_pred = resnet_model.predict(X_test)  # Predicted values
y_pred = y_pred.flatten()  # Flatten to 1D array

# Convert y_test to a numpy array
y_test_np = y_test.to_numpy()

# Set print options
np.set_printoptions(precision=2)

# Concatenate predictions and true values
result = np.concatenate((y_pred.reshape(-1, 1), y_test_np.reshape(-1, 1)), axis=1)
print(result[0:5])

75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[[451.82 455.27]
 [434.13 436.31]
 [431.65 440.68]
 [434.1  434.4 ]
 [475.17 482.06]]


Project Case Number 3: Check the performance of the two ANN and Deep Learning Networks

Import Dependencies

In [102]:
from sklearn.linear_model import LinearRegression, ARDRegression, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

Function For Evaluation

In [103]:
# Define a function for evaluation
def evaluate_model(model, X_train, y_train, X_test, y_test):
    """
    Train and evaluate the model. Prints MAE, RMSE, and R^2 scores.
    Args:
        model: The machine learning model to train.
        X_train, y_train: Training data.
        X_test, y_test: Test data.
    Returns:
        None, only print the evaluation metrics results
    """
    # Fit the model
    model.fit(X_train, y_train)

    # Predictions
    y_pred = model.predict(X_test)

    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print(f"{model.__class__.__name__}:")
    print(f"  MAE: {mae:.3f}")
    print(f"  RMSE: {rmse:.3f}")
    print(f"  R²: {r2:.3f}")
    print("-" * 40)

For Every algorithm evaluate the model

In [104]:
lin_reg = LinearRegression()
evaluate_model(lin_reg, X_train, y_train, X_test, y_test)

LinearRegression:
  MAE: 3.586
  RMSE: 4.537
  R²: 0.929
----------------------------------------


In [105]:
random_forest = RandomForestRegressor(n_estimators=10, random_state=42)
evaluate_model(random_forest, X_train, y_train, X_test, y_test)

RandomForestRegressor:
  MAE: 2.514
  RMSE: 3.560
  R²: 0.956
----------------------------------------


In [106]:
adr_regressor = ARDRegression()
evaluate_model(adr_regressor, X_train, y_train, X_test, y_test)

ARDRegression:
  MAE: 3.586
  RMSE: 4.537
  R²: 0.929
----------------------------------------


In [107]:
huber_regressor = HuberRegressor()
evaluate_model(huber_regressor, X_train, y_train, X_test, y_test)

HuberRegressor:
  MAE: 3.587
  RMSE: 4.563
  R²: 0.928
----------------------------------------


In [108]:
extra_trees_regressor = ExtraTreesRegressor()
evaluate_model(extra_trees_regressor, X_train, y_train, X_test, y_test)

ExtraTreesRegressor:
  MAE: 2.297
  RMSE: 3.297
  R²: 0.963
----------------------------------------


In [109]:
BaggingRegressor = BaggingRegressor()
evaluate_model(BaggingRegressor, X_train, y_train, X_test, y_test)

BaggingRegressor:
  MAE: 2.481
  RMSE: 3.511
  R²: 0.958
----------------------------------------


In [110]:
gradient_boost = GradientBoostingRegressor()
evaluate_model(gradient_boost, X_train, y_train, X_test, y_test)

GradientBoostingRegressor:
  MAE: 2.953
  RMSE: 3.914
  R²: 0.947
----------------------------------------


Ann Evaluation Since they are already trained

This is without epochs, optimization and loss function

In [111]:
y_pred_ann = ann.predict(X_test).flatten()
mae_ann = mean_absolute_error(y_test, y_pred_ann)
rmse_ann = np.sqrt(mean_squared_error(y_test, y_pred_ann))
r2_ann = r2_score(y_test, y_pred_ann)

print("Artificial Neural Network (ANN):")
print(f"  MAE: {mae_ann:.3f}")
print(f"  RMSE: {rmse_ann:.3f}")
print(f"  R²: {r2_ann:.3f}")
print("-" * 40)

75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Artificial Neural Network (ANN):
  MAE: 452.610
  RMSE: 452.904
  R²: -704.659
----------------------------------------


Rest Net Model Evaluation

In [112]:
y_pred_resnet = resnet_model.predict(X_test).flatten()
mae_resnet = mean_absolute_error(y_test, y_pred_resnet)
rmse_resnet = np.sqrt(mean_squared_error(y_test, y_pred_resnet))
r2_resnet = r2_score(y_test, y_pred_resnet)

print("Residual Network (ResNet):")
print(f"  MAE: {mae_resnet:.3f}")
print(f"  RMSE: {rmse_resnet:.3f}")
print(f"  R²: {r2_resnet:.3f}")
print("-" * 40)

75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Residual Network (ResNet):
  MAE: 4.539
  RMSE: 5.609
  R²: 0.892
----------------------------------------


Minor Changes in coding I want to create a structure like a classificaton report but I want it more function like so I will create another block of code to experiment will all the models

In [113]:
def evaluate_V2_model(model, X_train, y_train, X_test, y_test):
    """
    Train and evaluate the model. Returns MAE, RMSE, and R² scores. It is the Version 2 of the above evaluate function.
    Args:
        model: The machine learning model to train.
        X_train, y_train: Training data.
        X_test, y_test: Test data.
    Returns:
        dict: A dictionary containing MAE, RMSE, R². This is the structure like a classification report
    """
    # Fit the model
    model.fit(X_train, y_train)

    # Predictions
    y_pred = model.predict(X_test)

    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    return {
        'Model': model.__class__.__name__,
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2
    }

Create the results dictionary

In [114]:
results = []

Create the list of models I have chosen for the evalution I am using a list because I will append all the models performance. I am exclusing bagging regressor

In [115]:
models = [
    LinearRegression(),
    RandomForestRegressor(n_estimators=10, random_state=42),
    ARDRegression(),
    HuberRegressor(),
    ExtraTreesRegressor(),
    GradientBoostingRegressor()
]

Evaluate each model

In [116]:
for model in models:
    results.append(evaluate_model(model, X_train, y_train, X_test, y_test))

LinearRegression:
  MAE: 3.586
  RMSE: 4.537
  R²: 0.929
----------------------------------------
RandomForestRegressor:
  MAE: 2.514
  RMSE: 3.560
  R²: 0.956
----------------------------------------
ARDRegression:
  MAE: 3.586
  RMSE: 4.537
  R²: 0.929
----------------------------------------
HuberRegressor:
  MAE: 3.587
  RMSE: 4.563
  R²: 0.928
----------------------------------------
ExtraTreesRegressor:
  MAE: 2.328
  RMSE: 3.337
  R²: 0.962
----------------------------------------
GradientBoostingRegressor:
  MAE: 2.952
  RMSE: 3.911
  R²: 0.947
----------------------------------------


In [117]:
deep_learning_results = []

Evaluate the ANN Model add the results to dictionary

In [118]:
y_pred_ann = ann.predict(X_test).flatten()
mae_ann = mean_absolute_error(y_test, y_pred_ann)
rmse_ann = np.sqrt(mean_squared_error(y_test, y_pred_ann))
r2_ann = r2_score(y_test, y_pred_ann)
deep_learning_results.append({
    'Model': 'Artificial Neural Network (ANN)',
    'MAE': mae_ann,
    'RMSE': rmse_ann,
    'R²': r2_ann
})

75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


Evaluate the Renet Architecture model

In [119]:
y_pred_resnet = resnet_model.predict(X_test).flatten()
mae_resnet = mean_absolute_error(y_test, y_pred_resnet)
rmse_resnet = np.sqrt(mean_squared_error(y_test, y_pred_resnet))
r2_resnet = r2_score(y_test, y_pred_resnet)
deep_learning_results.append({
    'Model': 'Residual Network (ResNet)',
    'MAE': mae_resnet,
    'RMSE': rmse_resnet,
    'R²': r2_resnet
})

75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


Print the results of the Deep Learning

In [121]:
deep_learning_results_df = pd.DataFrame(deep_learning_results)
print(deep_learning_results_df)

                             Model         MAE        RMSE          R²
0  Artificial Neural Network (ANN)  452.609885  452.903751 -704.658968
1        Residual Network (ResNet)    4.538840    5.609174    0.891762
